# **멀티모달 LLM 사용하기**

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

## **라이브러리**

In [ ]:
import torch
import torchvision

print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("CUDA available?:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

In [ ]:
# ⚙️ 1. 패키지 설치 (필요 시)
# transformers(허깅페이스 라이브러리), pillow 설치가 아직 안 되어 있으면 아래 주석을 해제하고 실행하세요.
!pip install transformers==4.48.3 pillow

In [ ]:
# 설치된 transformers 버전 확인  -->(ex: Version: 4.55.4)
!pip list | grep -E 'transformers'



---



## **CLIP 기반 모델**
- CLIP(Contrastive Language-Image Pre-training, 대조학습 언어-이미지 사전 훈련 모델)
- 이미지와 텍스트의 임베딩을 모두 계산할 수 있는 임베딩 모델

### ✅ **예제 : CLIP으로 이미지에 맞는 캡션 선택하기**

- **모델명** : **openai/clip-vit-base-patch32**
- **총 파라미터**: 약 1.5억 (150M)
- **핵심 기능**: 이미지-텍스트 매칭 (Zero-Shot 분류, 이미지 검색 등)
    - **base**: 모델의 크기, 보통 'Small', 'Base', 'Large' 등으로 구분
    - **patch32** : 이미지를 32x32 픽셀 크기의 '패치(Patch)' 조각들로 나누어 입력으로 사용
    - 이 모델은 **제로샷 이미지 분류(Zero-Shot Image Classification**)에 가장 널리 사용됨
    - 모델에게 특정 이미지 한 장과 ["강아지 사진", "고양이 사진", "자동차 사진"] 같은 텍스트 후보들을 주면, 모델이 해당 이미지를 학습한 적이 없어도 어떤 텍스트 설명과 가장 유사한지 알려줄 수 있음

|구성 요소|	사용된 특정 모델|	파라미터|	학습 여부 (CLIP 기준)|	역할 및 특징|
|---|---|---|---|---|
|**1. Vision Encoder**(시각 인코더)|	ViT-Base (Patch 32)|	~8,800만 (88M)	|🔥 **Trained** (학습)|	이미지를 32x32 픽셀의 '패치(patch)'로 잘라 시퀀스로 만든 후, 이미지의 시각적 특징을 추출하여 **하나의 벡터(임베딩)**로 압축합니다.|
|**2. Text Encoder** (텍스트 인코더)|	Causal Transformer (BERT-like)|	~6,300만 (63M)	|🔥 **Trained**(학습)	|입력된 텍스트(문장)를 토큰화하고, 텍스트의 의미적 특징을 추출하여 **하나의 벡터(임베딩)**로 압축합니다.|
|(학습 목표)|	**Contrastive Learning**|	-|	-	|4억 개의 (이미지, 텍스트) 쌍 데이터를 사용하여, 1. '일치하는' 이미지-텍스트 벡터는 가깝게 (유사도 ↑) 2. '일치하지 않는' 쌍의 벡터는 멀게 (유사도 ↓) 만들도록 두 인코더를 함께 학습시킵니다.|

In [ ]:
# 라이브러리 가져오기
import torch
from PIL import Image
from torchvision import transforms
from transformers import CLIPTokenizerFast, CLIPImageProcessor, CLIPModel
# from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel

# 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")


# 사용할 CLIP 기반 임베딩 모델
model_name = "openai/clip-vit-base-patch32"

# 1) 이미지 전처기: Fast 버전 사용
clip_processor = CLIPImageProcessor.from_pretrained(model_name, use_fast=True)

# 2) 토크나이저: Fast 토크나이저 사용
clip_tokenizer = CLIPTokenizerFast.from_pretrained(model_name)

# 3) CLIP 기반 모델 정의:  Safetensors 포맷으로 가중치를 불러오도록 설정
model = CLIPModel.from_pretrained(
    model_name,
    use_safetensors=True    # 이 옵션을 추가하세요
).to(device)


# 텍스트 전처리하기
captions = [
    "a photo of a cat",
    "a photo of a dog",
    "a landscape with mountains"
]
# Fast 토크나이저를 이용해 토큰화 + 패딩
text_inputs = clip_tokenizer(
    captions,
    return_tensors="pt",
    padding=True,      # 길이 맞춤
    truncation=True    # 필요 시 자르기
)
input_ids = text_inputs.input_ids.to(device)
attention_mask = text_inputs.attention_mask.to(device)


# 이미지 전처리
image_path = "Golden-Retriever.jpg"
image = Image.open(image_path).convert("RGB")
image_inputs = clip_processor(
    images=image,
    return_tensors="pt"
)
pixel_values = image_inputs.pixel_values.to(device)
pixel_values.shape


image

In [ ]:
# 임베딩 생성하기
with torch.no_grad():
    # 이미지 임베딩 (batch_size=1, 512차원)
    image_embedding = model.get_image_features(pixel_values=pixel_values)

    # 텍스트 임베딩 (batch_size=len(texts), 512차원)
    text_embedding = model.get_text_features(
        input_ids=input_ids,
        attention_mask=attention_mask
    )

- **유사도 계산하기** (코사인 유사도)

In [ ]:
# 8. 유사도 계산 (코사인 유사도)
# 먼저 임베딩을 정규화(normalize)합니다.
image_embedding = image_embedding / image_embedding.norm(p=2, dim=-1, keepdim=True)
text_embedding = text_embedding / text_embedding.norm(p=2, dim=-1, keepdim=True)

# 코사인 유사도: image_embeds @ text_embeds.T
similarity = (image_embedding @ text_embedding.T).squeeze(0)  # shape: (len(texts),)
# CPU로 옮겨서 출력
similarity = similarity.cpu().tolist()

# 유사도 결과 출력
for text, score in zip(captions, similarity):
    print(f"‘{text}’ 유사도: {score:.4f}")

- **Zero-shot 분류하기**
    - 로짓 변환 사용
        - 로짓 변환(Logit Transformation)이란 0과 1 사이의 확률(p) 값을 을 이용해 실수 전체( $-\infty$ ~ $+\infty$) 범위의 값(로지트)으로 변환하는 수학적 함수
$log(\frac{p}{1-p})$, 로짓 변환으로 값을 증폭시켜 값을 강조함


In [ ]:
# 10. Zero-shot 분류 예제
# CLIPModel의 로지트(로그잇) 스코어를 사용하여 가장 유사한 문장 선택
with torch.no_grad():
    outputs = model(
        pixel_values=pixel_values,      # 이미지 전처리 결과
        input_ids=input_ids,            # 텍스트 전처리 결과
        attention_mask=attention_mask
    )
    logits_per_image = outputs.logits_per_image.squeeze(0)  # 이미지 → 텍스트 스코어
    probs = logits_per_image.softmax(dim=-1).cpu().tolist()
    print(logits_per_image)
    print(probs)

# 확률 기반으로 가장 높은 라벨 확인
best_idx = int(torch.argmax(logits_per_image))
print(f"\nZero-shot 분류 결과: '{captions[best_idx]}' (확률 {probs[best_idx]:.3f})")




---



## **BLIP 기반 모델**

- **BLIP** **Bootstrapping Language-Image Pre-training** for Unified Vision-Language Understanding and Generation
- "하나의 모델 아키텍처로 '이해(Understanding)'와 '생성(Generation)'을 모두 잘하면서, 동시에 '지저분한 웹 데이터(Noisy Data)'를 스스로 정제해서 학습 효율을 극대화하자!"

### ✅ **예제 : BLIP 기반 모델로 이미지에 자동 캡셔닝하기**
- **모델명** :  **Salesforce/blip-image-captioning-base**
- **총 파라미터**: 약 2.24억 (224M)
- **핵심 기능**: 이미지 캡션 생성 (이미지를 보고 문장 생성)


|구성 요소|	사용된 특정 모델|	파라미터|	학습 여부 (BLIP 기준)|	역할 및 특징|
|---|---|---|---|---|
|**1. Vision Encoder** (시각 인코더)|	ViT-Base (Patch 16)|	~8,600만 (86M)	|🔥 **Trained** (학습)	|이미지를 16x16 픽셀 패치로 나누어 시각 특징(feature)을 추출합니다. (BLIP-2와 달리, **BLIP-1은 이 인코더도 함께 학습**시킵니다.)|
|**2. Language Model** (언어 모델)|	BERT-base (MED)	|~1.38억 (138M)	|🔥 **Trained** (학습)|	(핵심) BERT-base 기반의 Encoder-Decoder 구조 (MED)입니다. 1. (**Encoder**): 이미지 특징과 텍스트를 함께 이해 (ITM) 2. (**Decoder**): 이미지 특징을 Cross-Attention으로 참조하며 텍스트를 '생성' (LM) 이 모델은 (2)번 기능(캡션 생성)에 특화되어 있습니다.|
|**(데이터)**|	**CapFilt** (정제 데이터)|	-	|-	|BLIP-1의 핵심 기여. 웹에서 수집한 노이즈 데이터를 자체 생성(Captioner) 및 필터링(Filter)하여 '**깨끗한 데이터'를 자동 생성**하여 학습에 사용했습니다.|

In [ ]:
import torch
from PIL import Image
import requests
from transformers import BlipProcessor, BlipForConditionalGeneration


# 1. 프로세서와 모델 로드 (이미지 캡셔닝용)
model_name = "Salesforce/blip-image-captioning-base"
blip_processor = BlipProcessor.from_pretrained(model_name)
model = BlipForConditionalGeneration.from_pretrained(model_name)


# 2. 이미지 준비 (BLIP-2와 동일한 이미지)
url = "https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg"
image = Image.open(requests.get(url, stream=True).raw)

image

In [ ]:
# 3. 이미지 캡션 생성 (LM 기능 테스트)

# (1) 조건 없이 생성 (Unconditional Captioning)
# 이미지만 입력으로 넣어줍니다.
inputs_uncond = blip_processor(images=image, return_tensors="pt")
out_uncond = model.generate(**inputs_uncond, max_new_tokens=50)
caption_uncond = blip_processor.decode(out_uncond[0], skip_special_tokens=True)

print("--- BLIP-1 이미지 캡셔닝 (조건 없음) ---")
print(f"생성된 캡션: {caption_uncond}")


# (2) 조건부 생성 (Conditional Captioning)
# "a man"으로 시작하는 캡션을 유도합니다.
text = "a man "
inputs_cond = blip_processor(images=image, text=text, return_tensors="pt")
out_cond = model.generate(**inputs_cond, max_new_tokens=50)
caption_cond = blip_processor.decode(out_cond[0], skip_special_tokens=True)

print("\n--- BLIP-1 이미지 캡셔닝 (조건부: 'a man') ---")
print(f"생성된 캡션: {caption_cond}")



---



## **BLIP-2**

### ✅ **예제 : BLIP-2 기반 모델로 이미지에 질문을 하자!(VQA)**
VQA(Visual Question Answering)

- BLIP-2 기반 모델 : **Salesforce/blip2-flan-t5-xxl**
    - BLIP-2 아키텍처를 사용하되,
    - LLM으로 Google의 'FLAN-T5-XXL' 모델을 탑재한,
        - 모델이 단순히 이미지를 묘사하는 것을 넘어 '지시(Instruction)'를 따르는 제로샷(Zero-Shot) VQA 성능이 좋은 모델(VQA 벤치마크에서 SOTA(State-of-the-Art)를 달성)
        - XXL의 의미 : FLAN-T5 계열 중에서도 110억 개(11B)의 파라미터를 가진 초거대 모델
    - Salesforce가 학습시킨 Vision-Language 모델

|구성 요소|	사용된 특정 모델|	파라미터|	학습 여부 (BLIP-2 기준)|	역할 및 특징|
|---|---|---|---|---|
|**1. Vision Encoder** (시각 인코더)|	CLIP ViT-G/14|	~10억 (1B)	|🧊 Frozen (동결)	|이미지를 보고 픽셀에서 복잡한 시각 특징(feature)을 추출합니다. (G/14는 매우 큰 ViT 모델입니다.)|
|**2. Q-Former** (시각-언어 번역기)|	BLIP-2 Q-Former|	~1.8억 (180M)|	🔥 Trained (학습)	|ViT가 추출한 방대한 시각 특징을 LLM이 이해할 수 있는 32개의 간결한 '시각 요약 벡터'로 압축/번역합니다.|
|**3. LLM** (언어 두뇌)|	FLAN-T5-XXL|	~110억 (11B)	|🧊 Frozen (동결)|	(핵심) Google이 T5 모델을 '지시(Instruction)' 데이터로 추가 학습시킨 모델. 단순 질의응답이 아닌, 프롬프트로 주어진 '지시'를 수행하는 데 특화되어 있습니다.|



In [ ]:
!pip install -U bitsandbytes accelerate transformers

In [ ]:
import torch
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BitsAndBytesConfig

# GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 양자화 설정 (최신 방식)
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8bit 양자화 활성화
    bnb_8bit_compute_dtype=torch.float16,  # 계산은 float16으로
    bnb_8bit_use_double_quant=True,  # 이중 양자화로 더욱 메모리 절약
    bnb_8bit_quant_type="nf4"  # 정규화된 4비트 양자화 타입
)

# 모델 로드
model_name = "Salesforce/blip2-flan-t5-xxl"
processor = Blip2Processor.from_pretrained(model_name)
model = Blip2ForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=quantization_config,  # 새로운 방식
    device_map="auto"  # GPU 자동 배치
)

# 이미지 준비
url = "http://images.cocodataset.org/val2017/000000039769.jpg"  # 자고 있는 고양이
image = Image.open(requests.get(url, stream=True).raw)

# 이미지 캡셔닝
inputs = processor(image, return_tensors="pt").to(device, torch.float16)
generated_ids = model.generate(**inputs, max_new_tokens=50)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

print(f"이미지 설명: {generated_text}")

image

In [ ]:
# 4. 프롬프트 준비 (FLAN-T5의 힘: '지시'를 내린다)

# 프롬프트 1: 단순 VQA
question1 = "What is in the image?"
prompt1 = f"Question: {question1} Answer:"

# 프롬프트 2: 고양이 숫자 세기
question2 = "How many cats are in the image?"
prompt2 = f"Question: {question2} Answer:"

# 프롬프트 3: 추론 (Reasoning) 지시
question3 = "What is the cat doing in the image?"
prompt3 = f"Question: {question3} Answer:"


print("--- BLIP-2 (FLAN-T5-XXL) 제로샷 테스트 ---")

# 5. 추론 실행 (gradient 계산 불필요)
with torch.no_grad():

    # --- 프롬프트 1 실행 ---
    inputs1 = processor(images=image, text=prompt1, return_tensors="pt").to(device, dtype=torch.float16)
    generated_ids1 = model.generate(**inputs1, max_new_tokens=20)
    answer1 = processor.batch_decode(generated_ids1, skip_special_tokens=True)[0].strip()
    print(f"질문 1: {question1}")
    print(f"답변 1: {answer1}")

    # --- 프롬프트 2 실행 ---
    inputs2 = processor(images=image, text=prompt2, return_tensors="pt").to(device, dtype=torch.float16)
    generated_ids2 = model.generate(**inputs2, max_new_tokens=10)
    answer2 = processor.batch_decode(generated_ids2, skip_special_tokens=True)[0].strip()
    print(f"\n질문 2: {question2}")
    print(f"답변 2: {answer2}")

    # --- 프롬프트 3 실행 ---
    inputs3 = processor(images=image, text=prompt3, return_tensors="pt").to(device, dtype=torch.float16)
    generated_ids3 = model.generate(**inputs3, max_new_tokens=20)
    answer3 = processor.batch_decode(generated_ids3, skip_special_tokens=True)[0].strip()
    print(f"\n질문 3: {question3}")
    print(f"답변 3: {answer3}")

### ✅ **예제 : 이미지 챗봇**
- 모델명 : Qwen/Qwen2-VL-2B-Instruct

In [ ]:
# ==========================================
# Qwen2-VL 한국어 이미지 채팅봇
# ==========================================

# 필수 라이브러리 설치
!pip install -q transformers accelerate qwen-vl-utils pillow torch

import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display, HTML
import ipywidgets as widgets

# ==========================================
# 1. 모델 및 프로세서 로드
# ==========================================
print("📦 모델을 로딩 중입니다...")

# GPU 메모리가 제한적인 경우 2B 모델 사용 권장
model_name = "Qwen/Qwen2-VL-2B-Instruct"  # 또는 "Qwen/Qwen2-VL-7B-Instruct"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # 메모리 절약
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_name)

print("✅ 모델 로딩 완료!")

# ==========================================
# 2. 이미지 처리 함수
# ==========================================
def load_image(image_source):
    """
    이미지를 URL 또는 로컬 파일에서 로드합니다.

    Parameters:
    - image_source: 이미지 URL 또는 로컬 파일 경로

    Returns:
    - PIL Image 객체
    """
    if image_source.startswith('http'):
        response = requests.get(image_source)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = Image.open(image_source).convert("RGB")
    return image

# ==========================================
# 3. 채팅 함수
# ==========================================
def chat_with_image(image, question, chat_history=[]):
    """
    이미지에 대한 질문에 답변하는 함수

    Parameters:
    - image: PIL Image 객체 또는 이미지 경로
    - question: 사용자 질문 (한국어)
    - chat_history: 이전 대화 기록

    Returns:
    - 모델의 답변
    """
    # 이미지가 경로인 경우 로드
    if isinstance(image, str):
        image = load_image(image)

    # 메시지 구성
    messages = []

    # 이전 대화 기록 추가
    for user_q, bot_a in chat_history:
        messages.append({
            "role": "user",
            "content": [
                {"type": "text", "text": user_q}
            ]
        })
        messages.append({
            "role": "assistant",
            "content": [
                {"type": "text", "text": bot_a}
            ]
        })

    # 현재 질문 추가 (첫 질문인 경우 이미지 포함)
    if len(chat_history) == 0:
        messages.append({
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": question}
            ]
        })
    else:
        messages.append({
            "role": "user",
            "content": [
                {"type": "text", "text": question}
            ]
        })

    # 입력 텍스트 준비
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    # 이미지와 비디오 정보 처리
    image_inputs, video_inputs = process_vision_info(messages)

    # 모델 입력 준비
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(model.device)

    # 텍스트 생성
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.9
        )

    # 생성된 토큰에서 입력 부분 제거
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

    # 텍스트로 디코딩
    output_text = processor.batch_decode(
        generated_ids_trimmed,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )[0]

    return output_text

# ==========================================
# 4. 대화형 위젯 인터페이스
# ==========================================
class ImageChatbot:
    def __init__(self):
        self.image = None
        self.chat_history = []
        self.setup_widgets()

    def setup_widgets(self):
        """위젯 설정"""
        # 이미지 업로드 위젯
        self.image_upload = widgets.FileUpload(
            accept='image/*',
            multiple=False,
            description='이미지 업로드'
        )
        self.image_upload.observe(self.on_image_upload, names='value')

        # 이미지 URL 입력
        self.image_url = widgets.Text(
            placeholder='또는 이미지 URL을 입력하세요',
            description='이미지 URL:',
            layout=widgets.Layout(width='80%')
        )
        self.url_button = widgets.Button(description='URL 로드')
        self.url_button.on_click(self.on_url_load)

        # 질문 입력
        self.question_input = widgets.Text(
            placeholder='이미지에 대해 질문하세요 (한국어)',
            description='질문:',
            layout=widgets.Layout(width='80%')
        )
        self.question_input.continuous_update = False
        self.question_input.observe(self.on_question_submit, names='value')

        # 출력 영역
        self.output = widgets.Output()

        # 초기화 버튼
        self.reset_button = widgets.Button(
            description='대화 초기화',
            button_style='warning'
        )
        self.reset_button.on_click(self.on_reset)

    def on_image_upload(self, change):
        """이미지 업로드 이벤트 핸들러"""
        if self.image_upload.value:
            uploaded_file = list(self.image_upload.value.values())[0]
            self.image = Image.open(BytesIO(uploaded_file['content'])).convert("RGB")
            self.chat_history = []  # 새 이미지면 대화 기록 초기화

            with self.output:
                self.output.clear_output()
                print("✅ 이미지가 업로드되었습니다!")
                display(self.image.resize((300, 300), Image.LANCZOS))

    def on_url_load(self, button):
        """URL 로드 버튼 이벤트 핸들러"""
        if self.image_url.value:
            try:
                self.image = load_image(self.image_url.value)
                self.chat_history = []  # 새 이미지면 대화 기록 초기화

                with self.output:
                    self.output.clear_output()
                    print("✅ 이미지가 로드되었습니다!")
                    display(self.image.resize((300, 300), Image.LANCZOS))
            except Exception as e:
                with self.output:
                    print(f"❌ 이미지 로드 실패: {e}")

    def on_question_submit(self, change):
        """질문 제출 이벤트 핸들러"""
        question = change['new']

        if not question:
            return

        if self.image is None:
            with self.output:
                print("⚠️ 먼저 이미지를 업로드하세요!")
            self.question_input.value = ""
            return

        # 질문 처리
        with self.output:
            display(HTML(f"<b>👤 사용자:</b> {question}"))
            print("🤔 답변 생성 중...")

        try:
            # 답변 생성
            answer = chat_with_image(self.image, question, self.chat_history)

            # 대화 기록에 추가
            self.chat_history.append((question, answer))

            # 결과 출력
            with self.output:
                display(HTML(f"<b>🤖 Qwen2-VL:</b> {answer}"))
                display(HTML("<br>"))

        except Exception as e:
            with self.output:
                print(f"❌ 오류 발생: {e}")

        # 입력창 초기화
        self.question_input.value = ""

    def on_reset(self, button):
        """초기화 버튼 이벤트 핸들러"""
        self.chat_history = []
        with self.output:
            self.output.clear_output()
            print("🔄 대화가 초기화되었습니다!")
            if self.image:
                display(self.image.resize((300, 300), Image.LANCZOS))

    def display(self):
        """위젯 표시"""
        url_box = widgets.HBox([self.image_url, self.url_button])

        display(widgets.VBox([
            widgets.HTML("<h2>🖼️ Qwen2-VL 한국어 이미지 채팅봇</h2>"),
            self.image_upload,
            url_box,
            widgets.HTML("<hr>"),
            self.question_input,
            self.reset_button,
            widgets.HTML("<hr>"),
            self.output
        ]))

# ==========================================
# 5. 챗봇 실행
# ==========================================
chatbot = ImageChatbot()
chatbot.display()